In [1]:
# Data cleaning notebook for Case Surveillance Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import stats
import warnings
warnings.simplefilter("ignore")
import statsmodels.api as sm

In [4]:
data = pd.read_csv("conditions.csv")
data.head()

,Data As Of,Start Date,End Date,Group,Year,Month,State,Condition Group,Condition,ICD10_codes,Age Group,COVID-19 Deaths,Number of Mentions,Flag
0,11/27/2022,01/01/2020,11/26/2022,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,1454.0,1520.0,NaN
1,11/27/2022,01/01/2020,11/26/2022,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,25-34,5666.0,5880.0,NaN
2,11/27/2022,01/01/2020,11/26/2022,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,35-44,14785.0,15395.0,NaN
3,11/27/2022,01/01/2020,11/26/2022,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,45-54,36798.0,38235.0,NaN
4,11/27/2022,01/01/2020,11/26/2022,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,55-64,80764.0,83685.0,NaN


### Data Preprocessing

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484380 entries, 0 to 484379
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Data As Of          484380 non-null  object 
 1   Start Date          484380 non-null  object 
 2   End Date            484380 non-null  object 
 3   Group               484380 non-null  object 
 4   Year                471960 non-null  float64
 5   Month               434700 non-null  float64
 6   State               484380 non-null  object 
 7   Condition Group     484380 non-null  object 
 8   Condition           484380 non-null  object 
 9   ICD10_codes         484380 non-null  object 
 10  Age Group           484380 non-null  object 
 11  COVID-19 Deaths     342695 non-null  float64
 12  Number of Mentions  347251 non-null  float64
 13  Flag                141685 non-null  object 
dtypes: float64(4), object(10)
memory usage: 51.7+ MB


### Dropping unnecessary columns

In [7]:
data = data.drop(['Data As Of','Number of Mentions','Flag'], axis=1)

In [8]:
data.head()

,Start Date,End Date,Group,Year,Month,State,Condition Group,Condition,ICD10_codes,Age Group,COVID-19 Deaths
0,01/01/2020,11/26/2022,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,1454.0
1,01/01/2020,11/26/2022,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,25-34,5666.0
2,01/01/2020,11/26/2022,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,35-44,14785.0
3,01/01/2020,11/26/2022,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,45-54,36798.0
4,01/01/2020,11/26/2022,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,55-64,80764.0


### Removing Null Values 

In [9]:
data.isna().sum()

Start Date              0
End Date                0
Group                   0
Year                12420
Month               49680
State                   0
Condition Group         0
Condition               0
ICD10_codes             0
Age Group               0
COVID-19 Deaths    141685
dtype: int64

In [10]:
data = data.dropna(subset=['Year','Month','COVID-19 Deaths'])

In [11]:
data.head()

,Start Date,End Date,Group,Year,Month,State,Condition Group,Condition,ICD10_codes,Age Group,COVID-19 Deaths
49680,01/01/2020,01/31/2020,By Month,2020.0,1.0,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,0.0
49681,02/01/2020,02/29/2020,By Month,2020.0,2.0,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,0.0
49682,03/01/2020,03/31/2020,By Month,2020.0,3.0,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,9.0
49683,04/01/2020,04/30/2020,By Month,2020.0,4.0,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,27.0
49684,05/01/2020,05/31/2020,By Month,2020.0,5.0,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,19.0


### Conversion of Data Type

In [13]:
data['Start Date'] = data['Start Date'].astype('datetime64')
data['End Date'] = data['End Date'].astype('datetime64')
data['Year'] = data['Year'].astype('int')
data['Month'] = data['Month'].astype('int')
data['COVID-19 Deaths'] = data['COVID-19 Deaths'].astype('int')

In [14]:
data.head()

,Start Date,End Date,Group,Year,Month,State,Condition Group,Condition,ICD10_codes,Age Group,COVID-19 Deaths
49680,2020-01-01,2020-01-31,By Month,2020,1,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,0
49681,2020-02-01,2020-02-29,By Month,2020,2,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,0
49682,2020-03-01,2020-03-31,By Month,2020,3,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,9
49683,2020-04-01,2020-04-30,By Month,2020,4,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,27
49684,2020-05-01,2020-05-31,By Month,2020,5,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,19


### Check for duplicate rows, if any

In [16]:
data = data.drop_duplicates()
data.head()

,Start Date,End Date,Group,Year,Month,State,Condition Group,Condition,ICD10_codes,Age Group,COVID-19 Deaths
49680,2020-01-01,2020-01-31,By Month,2020,1,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,0
49681,2020-02-01,2020-02-29,By Month,2020,2,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,0
49682,2020-03-01,2020-03-31,By Month,2020,3,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,9
49683,2020-04-01,2020-04-30,By Month,2020,4,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,27
49684,2020-05-01,2020-05-31,By Month,2020,5,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,19
